In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import os
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display, clear_output

In [117]:
# Load the CSV file
file_path = input('''🔴 Make sure you upload your file into the DRIVE 🔴
Enter your Drive File path : ''')
df = pd.read_csv(file_path)

# Print SMS
print("File Upload Successfully!✅")

# Display the first few rows to understand its structure
df.head()

🔴 Make sure you upload your file into the DRIVE 🔴
Enter your Drive File path : /content/drive/MyDrive/PFA/Bharatpur_Weather data_Processing/Bharatpur_Mandhera_19940101_20231231_027d39N_077d35E_LST.csv
File Upload Successfully!✅


,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,GWETTOP,1994,0.30,0.28,0.12,0.18,0.13,0.19,0.56,0.76,0.61,0.38,0.31,0.16,0.33
1,GWETTOP,1995,0.30,0.33,0.24,0.14,0.07,0.17,0.33,0.64,0.77,0.52,0.38,0.34,0.35
2,GWETTOP,1996,0.33,0.32,0.26,0.10,0.15,0.27,0.47,0.64,0.70,0.49,0.37,0.31,0.37
3,GWETTOP,1997,0.22,0.15,0.12,0.16,0.16,0.35,0.36,0.42,0.40,0.36,0.33,0.36,0.28
4,GWETTOP,1998,0.34,0.25,0.23,0.14,0.15,0.21,0.50,0.52,0.53,0.53,0.41,0.36,0.35


In [118]:
# Extract month columns automatically
month_cols = [col for col in df.columns if col not in ["PARAMETER", "YEAR"]]

# Create checkboxes for each month — all unchecked by default
month_checkboxes = []
for month in month_cols:
    cb = widgets.Checkbox(value=False, description=month)
    month_checkboxes.append(cb)

# Display checkboxes horizontally with wrapping
month_selection_ui = widgets.HBox(month_checkboxes, layout=widgets.Layout(flex_flow='row wrap'))
display(widgets.Label("📅 Select Months:"), month_selection_ui)

# print file path for input place
print(file_path)

# Create text input for place name
place_input = widgets.Text(
    value='',
    placeholder='Enter place name here',
    description='📍 Place:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)


# Create button to process
button = widgets.Button(description="✅ Process Data", button_style='success')
output = widgets.Output()



# Define button click event
def on_button_click(b):
    global df_pivot  # 👈 Make this accessible outside the function
    with output:
        clear_output()
        selected_months = [cb.description for cb in month_checkboxes if cb.value]
        place = place_input.value.strip()

        # Check the months and Print
        if not selected_months:
            print("⚠️ No months selected.")
            return
        if not place:
            print("⚠️ Please enter a place name.")
            return

        print(f"✅ Selected Months: {selected_months}")
        print(f"📍 Place: {place}")

        # Process the data
        df_melted = df.melt(id_vars=["PARAMETER", "YEAR"], var_name="Month", value_name="Value")
        df_melted = df_melted[df_melted["Month"].isin(selected_months)]
        df_melted["Month"] = df_melted["Month"].str.title()
        df_melted["Time"] = df_melted["Month"] + "-" + df_melted["YEAR"].astype(str)

        df_pivot = df_melted.pivot_table(index="Time", columns="PARAMETER", values="Value", aggfunc="first").reset_index()
        df_pivot.columns.name = None

        df_pivot["Year"] = df_pivot["Time"].str.split("-").str[1].astype(int)
        df_pivot["Month"] = pd.Categorical(df_pivot["Time"].str.split("-").str[0],
                                           categories=["Jan", "Feb", "Mar", "Apr", "May", "Jun",
                                                       "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                                           ordered=True)
        df_pivot = df_pivot.sort_values(by=["Year", "Month"]).drop(columns=["Year", "Month"])

        # Rename the processed columns name as required
        rename_dict = {
            "GWETTOP": "SSW",
            "PRECTOTCORR": "Rainfall",
            "QV2M": "Sp. Humidity",
            "RH2M": "Re. Humidity",
            "T2M_MAX": "Tmax",
            "T2M_MIN": "Tmin",
            "T2M_RANGE": "T_Range"
        }

        # Apply the rename names
        df_pivot = df_pivot.rename(columns=rename_dict)
        df_pivot["Place"] = place

        # Reorder columns
        df_pivot = df_pivot[["Place", "Time", "SSW", "Rainfall", "Re. Humidity", "Sp. Humidity", "Tmax", "Tmin", "T_Range"]]

        # Showing the Dataframe
        display(df_pivot)

# Bind the event
button.on_click(on_button_click)

# Display inputs and output
display(place_input, button, output)

Label(value='📅 Select Months:')

/content/drive/MyDrive/PFA/Bharatpur_Weather data_Processing/Bharatpur_Mandhera_19940101_20231231_027d39N_077d35E_LST.csv


Text(value='', description='📍 Place:', layout=Layout(width='50%'), placeholder='Enter place name here', style=…

Button(button_style='success', description='✅ Process Data', style=ButtonStyle())

Output()

In [119]:
# Save to CSV (only if df_pivot is already created)

# Provide Your Folder path for output data 👇
output_path = "/content/drive/MyDrive/PFA_Result/Bharatpur/"

# input fileName :
file_name = f"{df_pivot['Place'].iloc[0]}_Processed_Data.csv"

# Download as CSV:
df_pivot.to_csv(output_path + file_name, index=False)
print(f"📁 Data saved to: {output_path}{file_name} successfully✔️") # Showing the successfull sms.

📁 Data saved to: /content/drive/MyDrive/PFA_Result/Bharatpur/Bharatpur_Mandhera_Processed_Data.csv successfully✔️
